In [ ]:
%cd /home/stefano/dev/active/spreads-arb
%load_ext line_profiler

import itertools
import logging
import os
import pickle
from datetime import date, datetime, timedelta

import cryptomart as cm
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import requests
import vectorbt as vbt

import argparse
import os
import pickle

import cryptomart as cm
import pandas as pd
from app.data_prep.dummy import dummy_bid_ask_spreads
from app.data_prep.real import all_bid_ask_spreads, all_funding_rates, all_ohlcv, get_fee_info
from app import util
from app.feeds import Spread

import app

identifier = "spreads-arb-v2"

# spreads = pd.read_pickle("/home/stefano/dev/active/spreads-arb/data/granular_spreads.pkl")
# spread = pickle.loads(spreads.iloc[0].spread)
# res = app.BacktestRunner(
#     log_dir=os.path.join(
#         os.getenv("ACTIVE_DEV_PATH", "/home/stefano/development/active_dev"), "logs", "spreads-arb-v2"
#     ),
#     vbt_function=app.vbt.chained_quotes,
#     use_slippage=False,
#     use_funding_rate=False,
#     profitable_only=True,
#     z_score_thresholds=(0, 1),
#     z_score_period=30,
# ).run_quotes(spreads)

In [ ]:
spread.zscore(column="mid_price", period=30).head(200)

In [ ]:

leg_0_bid = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("bid_price").T.iloc[0], axis=1)
)
leg_1_bid = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("bid_price").T.iloc[1], axis=1)
)
bid_prices = np.concatenate([leg_0_bid, leg_1_bid], axis=0).T
bid_prices = np.vstack((np.full_like(bid_prices[:29], np.nan), bid_prices))
        
leg_0_ask = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("ask_price").T.iloc[0], axis=1)
)
leg_1_ask = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("ask_price").T.iloc[1], axis=1)
)
ask_prices = np.concatenate([leg_0_ask, leg_1_ask], axis=0).T
ask_prices = np.vstack((np.full_like(ask_prices[:29], np.nan), ask_prices))
        
leg_0_bid_sz = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("bid_amount").T.iloc[0], axis=1)
)
leg_1_bid_sz = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("bid_amount").T.iloc[1], axis=1)
)
bid_sizes = np.concatenate([leg_0_bid_sz, leg_1_bid_sz], axis=0).T
bid_sizes = np.vstack((np.full_like(bid_sizes[:29], np.nan), bid_sizes))
        
leg_0_ask_sz = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("ask_amount").T.iloc[0], axis=1)
)
leg_1_ask_sz = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("ask_amount").T.iloc[1], axis=1)
)
ask_sizes = np.concatenate([leg_0_ask_sz, leg_1_ask_sz], axis=0).T
ask_sizes = np.vstack((np.full_like(ask_sizes[:29], np.nan), ask_sizes))


leg_0_mid = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("mid_price").T.iloc[0], axis=1)
)
leg_1_mid = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("mid_price").T.iloc[1], axis=1)
)
mid_prices = np.concatenate([leg_0_mid, leg_1_mid], axis=0).T
mid_prices = np.vstack((np.full_like(mid_prices[:29], np.nan), mid_prices))

zscore = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).zscore("bid_price", 30), axis=1)
).T

In [ ]:
bid_prices.nbytes / 1024 / 1024

In [ ]:
bid_prices.shape

In [ ]:
current_zscore = zscore[593]

In [ ]:
direction = np.where(np.abs(current_zscore) > 1, -np.sign(current_zscore), 0)

In [ ]:
direction

In [ ]:
ask_price = ask_prices[593]
ask_size = ask_sizes[593]
bid_price = bid_prices[593]
bid_size = bid_sizes[593]

In [ ]:
leg_0_sizes = np.where(direction == 1, bid_size[:100], ask_size[:100])
leg_1_sizes = np.where(direction == 1, bid_size[100:], ask_size[100:])

In [ ]:
leg_0_sizes

In [ ]:
sizes = np.vstack((leg_0_sizes, leg_1_sizes)).T
sizes.shape

In [ ]:
leg_1_sizes.shape

In [ ]:
ask_price.shape

In [ ]:
from numba import njit



@njit
def foo(x, y):
    if x is not None:
        print(1)

foo(None, 2)

In [ ]:
leg_0_bid = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("bid_price").T.iloc[0], axis=1)
)
leg_1_bid = np.array(
    spreads.apply(lambda s: pickle.loads(s.spread).underlying_col("bid_price").T.iloc[1], axis=1)
)
bid_prices = np.concatenate([leg_0_bid, leg_1_bid], axis=0).T
    

In [ ]:
bid_prices.shape

In [ ]:
np.full_like(bid_prices[:29], np.nan).shape

In [ ]:
np.vstack((np.full_like(bid_prices[:29], np.nan), bid_prices))

In [ ]:
np.insert(bid_prices, 0, np.full_like(bid_prices[:29], np.nan))

In [ ]:

quotes = app.data_prep.all_tardis_quotes()
quotes

In [ ]:

for idx, row in quotes.iterrows():
    exchange, inst_type, symbol = idx
    daily_close = app.data_prep.real.cm_client.ohlcv(exchange, symbol, inst_type, "2023-03-12", "2023-04-10")
    if daily_close.close.isna().any():
        quotes.drop(idx, inplace=True)
        continue
        
    q = pickle.loads(row.quotes).set_index("timestamp")
    q = q.iloc[:9000]
    if q.mid_price.isna().any():
        quotes.drop(idx, inplace=True)
        continue
    
    combined_prices = pd.concat([q.mid_price, daily_close.set_index("open_time").close], axis=0).sort_index()
    q = q.reindex(combined_prices.index)
    q["mid_price"] = combined_prices
    q.index.name = "timestamp"
    q = q.reset_index()
    quotes.at[idx, "quotes"] = pickle.dumps(q)
        

In [ ]:
pickle.loads(quotes.iloc[0].quotes).set_index("timestamp").mid_price.plot()

In [ ]:
pickle.loads(quotes.iloc[0].quotes).zscore().plot()

In [ ]:
pickle.loads(quotes.iloc[0].quotes)

In [ ]:
fee_info = app.data_prep.get_fee_info(refresh=False, identifiers=[identifier])
fee_info

In [ ]:
instrument_data = quotes.join(fee_info).reorder_levels(quotes.index.names)
instrument_data

In [ ]:
instrument_data_crossed = (
    instrument_data.reset_index()
    .merge(instrument_data.reset_index(), how="cross", suffixes=("_a", "_b"))
    .pipe(
        lambda df: df[
            (df.exchange_a < df.exchange_b) & (df.inst_type_a <= df.inst_type_b) & (df.symbol_a == df.symbol_b)
        ]
    )
    .drop(columns="symbol_b")
    .rename(columns={"symbol_a": "symbol"})
    .reset_index(drop=True)
)
del instrument_data
instrument_data_crossed


In [ ]:
s2 = Spread.from_api("1000LUNC", ["binance", "bybit"], "2023-04-10", "2023-05-10")

In [ ]:
s = Spread.from_quotes(pickle.loads(instrument_data_crossed.iloc[0].quotes_a), pickle.loads(instrument_data_crossed.iloc[0].quotes_b))

In [ ]:

@util.cached("/tmp/cache/spread_volatility", refresh=False)
def load_volatility():
    volatility = pd.DataFrame(
        index=pd.MultiIndex.from_arrays(
            [[], [], [], [], []], names=["exchange_a", "exchange_b", "inst_type_a", "inst_type_b", "symbol"]
        )
    )
    
    for idx, row in instrument_data_crossed.iterrows():
        indexer = (row.exchange_a, row.exchange_b, row.inst_type_a, row.inst_type_b, row.symbol)
        volatility.at[indexer, "volatility"] = Spread.from_quotes(pickle.loads(row.quotes_a), pickle.loads(row.quotes_b)).volatility(column="bid_price")
        
    return volatility

volatility = load_volatility()
volatility.sort_values("volatility", ascending=False).head(20)

In [ ]:
most_volatile = volatility.sort_values("volatility", ascending=False).head(10)

# filtered_idx = (
#     most_volatile.reset_index()
#     .melt(id_vars=["symbol"], value_vars=["exchange_a", "exchange_b"])
#     .drop_duplicates()
#     .assign(inst_type="perpetual")
#     .rename(columns={"value": "exchange"})[["exchange", "inst_type", "symbol"]]
#     .values.tolist()
# )

# quotes_filtered = quotes.loc[filtered_idx]
# del quotes
# quotes_filtered

In [ ]:
instrument_data_crossed = instrument_data_crossed.merge(most_volatile, right_index=True, left_on=["exchange_a", "exchange_b", "inst_type_a", "inst_type_b", "symbol"])

In [ ]:
instrument_data_crossed.to_pickle("/home/stefano/dev/active/spreads-arb/data/instrument_data_crossed.pkl")

In [ ]:
instrument_data_crossed = pd.read_pickle("/home/stefano/dev/active/spreads-arb/data/instrument_data_crossed.pkl")

In [ ]:
spreads = pd.DataFrame(
    index=pd.MultiIndex.from_arrays(
        [[], [], [], [], []], names=["exchange_a", "exchange_b", "inst_type_a", "inst_type_b", "symbol"]
    )
)

for idx, row in instrument_data_crossed.iterrows():
    indexer_a = (row.exchange_a, row.inst_type_a, row.symbol)
    indexer_b = (row.exchange_b, row.inst_type_b, row.symbol)
    indexer = (row.exchange_a, row.exchange_b, row.inst_type_a, row.inst_type_b, row.symbol)
    spread = Spread.from_quotes(pickle.loads(row.quotes_a), pickle.loads(row.quotes_b))

    spreads.at[indexer, "spread"] = pickle.dumps(spread)
    spreads.at[
        indexer, "alias"
    ] = f"{row.exchange_a[:4]}_{row.exchange_b[:4]}_{row.inst_type_a[:4]}_{row.inst_type_b[:4]}_{row.symbol}"
    spreads.at[indexer, "volatility"] = spread.volatility(column="bid_price")
    spreads.at[indexer, "earliest_time"] = spread.earliest_time
    spreads.at[indexer, "latest_time"] = spread.latest_time
    spreads.at[indexer, "valid_rows"] = len(spread.valid_rows)
    spreads.at[indexer, "missing_rows"] = len(spread.missing_rows)
    spreads.at[indexer, "gaps"] = spread.gaps

    fee_info_keys = ["init_margin", "maint_margin", "fee_pct", "fee_fixed"]
    for key in ["a", "b"]:
        spreads.at[indexer, f"fee_info_{key}"] = pickle.dumps(
            {k: getattr(row, f"{k}_{key}") for k in fee_info_keys}
        )


In [ ]:
spreads.to_pickle("/home/stefano/dev/active/spreads-arb/data/granular_spreads.pkl")

In [ ]:
spreads = pd.read_pickle("/home/stefano/dev/active/spreads-arb/data/granular_spreads.pkl")
spread = pickle.loads(spreads.iloc[0].spread)

In [ ]:
spreads

In [ ]:
spread.set_index("timestamp").mid_price.plot()

In [ ]:
spread = pickle.loads(spreads.iloc[0].spread)
pd.DataFrame(spread.set_index("timestamp").underlying_col("mid_price")).iloc[30:].plot()

In [ ]:
pd.DataFrame(spread.set_index("timestamp").underlying_col("bid_amount")).iloc[30:].plot()

In [ ]:
spread.iloc[30:]

In [ ]:
quotes.loc["bitmex"]

In [ ]:
spread.ohlcv_list[0].exchange_name

In [ ]:
spread.mid_price.iloc[30:].plot()

In [ ]:
spread.set_index("timestamp").mid_price.head(20)

In [ ]:
spread.set_index("timestamp").mid_price.rolling("30D").mean().reset_index(drop=True).head(200).plot()
spread.set_index("timestamp").mid_price.head(200).reset_index(drop=True).plot()

In [ ]:
spread.zscore("mid_price", period="30D").reset_index(drop=True).head(200).plot()

In [ ]:
spread.zscore("mid_price", period=pd.Timedelta("30D"))

In [ ]:
spread.zscore(column="mid_price").iloc[30:].plot()